In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../src")
import feature.experimental_dataset as experimental_dataset
import feature.simulated_dataset as simulated_dataset
import feature.util as feature_util
import numpy as np
import tqdm.notebook

In [2]:
peaks_bed_path = "/home/tsenga5/mechint_regnet/data/encode/chipseq/ENCSR607XFI_CTCF_HepG2/ENCFF664UGR_idrpeaks.bed.gz"

genome_fasta_path = "/home/tsenga5/mechint_regnet/data/genomes/hg38.fa"
chrom_sizes_path = "/home/tsenga5/mechint_regnet/data/genomes/GRCh38_EBV.chrom.sizes.tsv"
batch_size = 128
input_length = 500
input_center_overlap = 400
negative_ratio = 1
revcomp = True
num_workers = 2
data_seed = 123

In [3]:
data_loader_1 = experimental_dataset.create_data_loader(
    peaks_bed_path, genome_fasta_path, chrom_sizes_path, batch_size,
    input_length, input_center_overlap, negative_ratio, revcomp, num_workers,
    data_seed, chrom_set=["chr21"], return_coords=True
)
data_loader_2 = experimental_dataset.create_data_loader(
    peaks_bed_path, genome_fasta_path, chrom_sizes_path, batch_size,
    input_length, input_center_overlap, negative_ratio, revcomp, num_workers,
    data_seed, chrom_set=["chr21"], return_coords=True
)

/home/tsenga5/miniforge3/envs/mechint/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/tsenga5/miniforge3/envs/mechint/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_m

In [4]:
num_epochs = 3
all_coords, all_seqs = [], []
for data_loader in (data_loader_1, data_loader_2):
    coords, seqs = [], []
    for _ in range(num_epochs):
        data_loader.dataset.on_epoch_start()
        print(data_loader.dataset.rng.bit_generator.state)
        for batch in tqdm.notebook.tqdm(data_loader):
            coords.extend([("%s:%d-%d" % tuple(row)) for row in batch[2]])
            seqs.extend(feature_util.one_hot_to_seqs(batch[0]))
    all_coords.append(coords)
    all_seqs.append(seqs)

{'bit_generator': 'PCG64', 'state': {'state': 98423212233957053049683177625231049430, 'inc': 17686443629577124697969402389330893883}, 'has_uint32': 1, 'uinteger': 1597002389}


  0%|          | 0/22 [00:00<?, ?it/s]

{'bit_generator': 'PCG64', 'state': {'state': 284875759728818632356215597596530428350, 'inc': 17686443629577124697969402389330893883}, 'has_uint32': 1, 'uinteger': 3970679553}


  0%|          | 0/22 [00:00<?, ?it/s]

{'bit_generator': 'PCG64', 'state': {'state': 280792087133618858882770275075456899387, 'inc': 17686443629577124697969402389330893883}, 'has_uint32': 1, 'uinteger': 1836400246}


...

  0%|          | 0/22 [00:00<?, ?it/s]

{'bit_generator': 'PCG64', 'state': {'state': 284875759728818632356215597596530428350, 'inc': 17686443629577124697969402389330893883}, 'has_uint32': 1, 'uinteger': 3970679553}


  0%|          | 0/22 [00:00<?, ?it/s]

{'bit_generator': 'PCG64', 'state': {'state': 280792087133618858882770275075456899387, 'inc': 17686443629577124697969402389330893883}, 'has_uint32': 1, 'uinteger': 1836400246}


  0%|          | 0/22 [00:00<?, ?it/s]

In [5]:
print("Coordinates generated by loader 1: %d" % len(set(all_coords[0])))
print("Coordinates generated to loader 2: %d" % len(set(all_coords[1])))
print("Coordinates shared between loader 1 and loader 2: %d" % len(set(all_coords[0]) & set(all_coords[1])))
print()
print("Sequences generated by loader 1: %d" % len(set(all_seqs[0])))
print("Sequences generated to loader 2: %d" % len(set(all_seqs[1])))
print("Sequences shared between loader 1 and loader 2: %d" % len(set(all_seqs[0]) & set(all_seqs[1])))

Coordinates generated by loader 1: 4168
Coordinates generated to loader 2: 4168
Coordinates shared between loader 1 and loader 2: 4168

Sequences generated by loader 1: 7725
Sequences generated to loader 2: 7725
Sequences shared between loader 1 and loader 2: 7725
